In [44]:
%graph_notebook_version

3.4.1


In [47]:
%disable_debug

In [2]:
%%graph_notebook_config
{
  "host": "neptune-cluster-poc-identity-graph.cluster-c4k0tumhelmt.us-east-2.neptune.amazonaws.com",
  "port": 8182,
  "auth_mode": "IAM",
  "load_from_s3_arn": "",
  "ssl": true,
  "aws_region": "us-east-2",
  "sparql": {
    "path": "sparql"
  },
  "gremlin": {
    "traversal_source": "g"
  }
}

set notebook config to:
{
  "host": "neptune-cluster-poc-identity-graph.cluster-c4k0tumhelmt.us-east-2.neptune.amazonaws.com",
  "port": 8182,
  "auth_mode": "IAM",
  "load_from_s3_arn": "",
  "ssl": true,
  "aws_region": "us-east-2",
  "sparql": {
    "path": "sparql"
  },
  "gremlin": {
    "traversal_source": "g"
  }
}


In [48]:
%graph_notebook_config

{
  "host": "neptune-cluster-poc-identity-graph.cluster-c4k0tumhelmt.us-east-2.neptune.amazonaws.com",
  "port": 8182,
  "auth_mode": "IAM",
  "load_from_s3_arn": "",
  "ssl": true,
  "aws_region": "us-east-2",
  "sparql": {
    "path": "sparql"
  },
  "gremlin": {
    "traversal_source": "g"
  }
}


In [406]:
%status

{'status': 'healthy',
 'startTime': 'Tue Jun 21 18:11:45 UTC 2022',
 'dbEngineVersion': '1.1.1.0.R3',
 'role': 'writer',
 'dfeQueryEngine': 'viaQueryHint',
 'gremlin': {'version': 'tinkerpop-3.5.2'},
 'sparql': {'version': 'sparql-1.1'},
 'opencypher': {'version': 'Neptune-9.0.20190305-1.0'},
 'labMode': {'ObjectIndex': 'disabled',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'enabled',
  'Streams': 'disabled',
  'AuditLog': 'enabled'},
 'settings': {'clusterQueryTimeoutInMs': '60000'}}

In [407]:
%load

Button(description='Submit', style=ButtonStyle())

Output()

### Exploring the Identity Graph
In order to better understand data model and schema for the graph we can execute the below graph queries to know common entities like nodes/vertices and relationships/edges

#### Amount of nodes/vertices group by label

In [451]:
%%gremlin
g.V().groupCount().by(label).unfold()

#### Amount of relationships/edges group by label

In [452]:
%%gremlin
g.E().groupCount().by(label).unfold()

#### Top 10 of Device IDs by amount of known users linked

In [493]:
%%gremlin
g.V().hasLabel('DeviceID').
    project('device_id','known_users_linked').
        by(id).
        by(in('usedDevice').out('loggedAs').count()).
    order().
        by('known_users_linked',desc).
    limit(10)

#### Top 10 of Client IPs by amount of known users linked

In [494]:
%%gremlin
g.V().hasLabel('ClientIP').
    project('client_ip','known_users_linked').
        by(id).
        by(in('lastSeenAtIP').out('loggedAs').count()).
    order().
        by('known_users_linked',desc).
    limit(10)

### Find out information about user interests based on the activity of the user across all devices
Suppose you are hosting a web platform and collecting clickstream data as users browse your site or use your mobile app. In the majority of situations, users using your platform will be anonymous (or non-registered or logged in users). However, these anonymous users may be linked to other known users in that have used our platform before. We can join (or resolve) the identity of the anonymous user with attributes we know about existing users to make some assumptions (based off of known user behavior and heuristics) in order to know more about this anonymous user. We can then use this information to target the user with advertising, special offers, discounts, etc

Let's use an example where we have an anonymous user interaction through AnyCompany Marketplace website (e.i. web session). The user's interaction is registered as session id **'c3e8fa97-0f87-4d51-91f9-46514d191277'**. We want to know more about this user and if it is linked to other users on our platform. This anonymous user is considered a **"transient ID"** in our graph data model. Assuming this user does not have a link to a known user, or **"persistent ID"**, how might we find connections from this transient ID to other known user IDs?

Looking at the data model, you can see that **"SessionID"** (a.k.a. transient ID) vertices in our graph are connected to **"ClientIP"** vertices by an outgoing edge. We can traverse across "ClientIP" vertices to get to other linked "SessionID" that might be linked to a known user (a.k.a. persistent ID). Let's do that in the following graph query.

In [495]:
%%gremlin
g.V('dcb08831-26fd-490f-9264-44ca763e33b1').
    out('lastSeenAtIP').
    in('lastSeenAtIP').
    out('loggedAs').
    dedup().
    path()

Let's use another example where we have an anonymous user interaction through AnyCompany's Mobile Application. The user's interaction is coming from a device with id **'CFE9EA5D070F2557'**. We want to know more about this user and if it is linked to other users on our platform. This anonymous user is considered a "transient ID" in our graph data model. Assuming this user does not have a link to a known user, or "persistent ID", how might we find connections from this transient ID to other known user IDs?

Looking at the data model, you can see that **"DeviceID"** vertices in our graph are connected to **"SessionID"** (a.k.a. transient ID) vertices by an incomming edge. We can traverse across "SessionID" vertices that might be linked to a known user (a.k.a. persistent ID) and are connected to same "DeviceID" (CFE9EA5D070F2557). Let's do that in the following graph query.

In [496]:
%%gremlin
g.V('CFE9EA5D070F2557').
    in('usedDevice').
    out('loggedAs').
    dedup().
    out('hasEmail','hasExternalId').
    out('hasPurchased').
    path()

Let's use another example where we have an user interaction at the register of an AnyCompany's physical store. The user's interaction is registered with the customer external ID **'709-31-6436'** provided by the user at the register. We want to know more about this user and its purchase history, even if the user is linked to other known users on our platform.

Looking at the data model, you can see that **"ExternalID"** vertices in our graph are connected to **"User"** (a.k.a. persistent ID) vertices by an incomming edge. We can traverse across "User" vertices to get other linked purchase identifiers (e.g. Email) that might have been used by same user when purchasing other products. Let's do that in the following graph query.

In [497]:
%%gremlin
g.V('709-31-6436').
    in('hasExternalId').
    out().
    out('hasPurchased').
    path()